In [1]:
import os
import base64
import json
import re
import requests
from dotenv import load_dotenv
from tqdm import tqdm
from langchain_community.document_loaders import PyPDFLoader
from pdf2image import convert_from_path
import openai

# Cargar las variables de entorno desde el archivo .env
load_dotenv(r'C:\Users\jon.ruizcarrillo\Desktop\Projects\TFM\inno-dashboard-assistant-2024\.env')

# Obtener la clave de API desde las variables de entorno
api_key = os.getenv("OPENAI_API_KEY")
openai.api_key = api_key  # Configurar la clave de API para OpenAI

# Función para codificar la imagen en base64
def encode_image(image_path):
    with open(image_path, "rb") as image_file:
        return base64.b64encode(image_file.read()).decode('utf-8')

# Función para convertir PDF en imágenes (una por página)
def pdf_to_images(pdf_path, output_dir):
    images = convert_from_path(pdf_path)
    image_paths = []
    for i, image in enumerate(images):
        image_path = os.path.join(output_dir, f"page_{i+1}.jpg")
        image.save(image_path, 'JPEG')
        image_paths.append(image_path)
    return image_paths

def create_image_messages(image_path, report_id, page_number):
    messages = [{
        "role": "user",
        "content": [
            {
                "type": "text",
                "text": (
                    "I have exported the following dashboard as image. The image corresponds to a dashboard with key metrics **KPIs** and **visualizations**, and **table values**. "
                    "Your task is to identify **ALL** the **KPIs**, **visualizations**, and **table values** shown in each image, providing their exact titles, values, and any other relevant information visible. "
                    "Additionally, you should provide insights, conclusions, or analysis of the data as part of your response. "
                    "If any table is incomplete, please return a warning indicating which tables were not fully captured including the word warning.\n"
                    "Return the final information in JSON format with this structure:\n"
                    "{\n"
                    "  'report_id': '{report_id}',\n"
                    "  'date': '2024-09-23',\n"
                    "  'page': {page_number}\n"                    
                    "  'detected_elements': {\n"
                    "      'KPIs': [\n"
                    "        {\n"
                    "          'kpi_title': '<Exact KPI title>',\n"
                    "          'kpi_value': '<KPI value>',\n"
                    "          'kpi_reference': '<KPI reference or baseline>',\n"
                    "          'insights': '<Any key insights from this KPI>'\n"
                    "        },\n"
                    "        ...\n"
                    "      ],\n"
                    "      'charts': [\n"
                    "        {\n"
                    "          'visualization_title': '<Exact visualization title>',\n"
                    "          'visualization_type': '<Bar Chart, Line Chart, Pie Chart, etc.>',\n"
                    "          'metrics_displayed': ['<List of metrics in the chart>'],\n"
                    "          'values': ['<Values from the visualization>'],\n"
                    "          'insights': '<Any key insights from this visualization>'\n"
                    "        },\n"
                    "        ...\n"
                    "      ],\n"
                    "      'tables': [\n"
                    "        {\n"
                    "          'table_title': '<Exact table title>',\n"
                    "          'columns': ['<List of columns>'],\n"
                    "          'rows': ['<List of all rows>'\n"
                    "            ['<Row 1 values>'],\n"
                    "            ['<Row 2 values>'],\n"
                    "            ...\n"
                    "          ],\n"
                    "          'insight': 'Detailed analysis of the table.'\n"
                    "        },\n"
                    "        ...\n"
                    "      ],\n"
                    "  'final_conclusions': 'Summary and final conclusions based on the analysis of KPIs, charts, and tables.',\n"
                    "  },\n"
                    "}\n"
                ).replace("{report_id}", report_id).replace("{page_number}", str(page_number))
            }
        ]
    }]

    # Cargar solo una imagen en lugar de un bucle
    base64_image = encode_image(image_path)
    messages[0]["content"].append({
        "type": "image_url",
        "image_url": {
            "url": f"data:image/jpeg;base64,{base64_image}",
            "detail": "high"
        }
    })

    return messages


def process_image(image_path, report_id, page_number):
    """Procesa las imágenes y devuelve la respuesta de la API."""
    messages = create_image_messages(image_path, report_id, page_number)
    headers = {
        "Content-Type": "application/json",
        "Authorization": f"Bearer {api_key}"
    }
    payload = {
        "model": "gpt-4o-mini",
        "messages": messages,
        "max_tokens": 15000
    }
    
    response = requests.post("https://api.openai.com/v1/chat/completions", headers=headers, json=payload)
    return response.json()

def process_text_with_openai(page_content, report_id, page_number, processed_content=None):
    """Procesa el contenido de una página en formato de texto."""
    
    # Crea el contenido del mensaje incluyendo el contenido de la página y los resultados procesados
    processed_content_text = processed_content if processed_content else "No processed content available."
    
    messages = [{
        "role": "user",
        "content": (
            f"I have exported the following dashboards as PDF corresponding to report '{report_id}' on page {page_number}. "
            f"Here is the content in plain text extracted from the pdf: {page_content}\n"
            f"Here are the previously processed details from pdf converted to jpeg images (image processor): {processed_content_text}\n"
            "Your task is to identify all the missing values and complete the JSON file provided by the image processor."
            "Ensure to maintain the structure received from the image processor and fill in the KPI titles, values, visualization titles, displayed metrics, and any insights exactly as they appear in the image."
        )
    }]
    
    client = openai.OpenAI()

    try:
        # Llamar a la API
        completion = client.chat.completions.create(
            model="gpt-4o-mini",
            messages=messages,
            max_tokens=15000,
            temperature=0.3,
        )

        response_content = completion.choices[0].message.content.strip()
        return response_content
    
    except Exception as e:
        print(f"Error al llamar a la API de OpenAI: {e}")
        return None


def pdf_to_text(pdf_path):
    """Convierte todas las páginas de un PDF a texto y devuelve una lista de los contenidos por página."""
    loader = PyPDFLoader(pdf_path, extraction_mode="layout")
    pages = loader.lazy_load()

    pdf_text_contents = []
    for idx, doc in enumerate(pages):
        pdf_text_contents.append(doc.page_content)  # Almacena el contenido de cada página como un string

    return pdf_text_contents


    
    return results

def main(pdf_path, output_dir):
    """Función principal que coordina el procesamiento de imágenes y texto del PDF."""
    report_id = os.path.basename(pdf_path).replace('.pdf', '')  # Nombre del archivo sin extensión

    # Convertir el PDF a imágenes
    image_paths = pdf_to_images(pdf_path, output_dir)

    # Convertir todo el PDF a texto
    pdf_text_contents = pdf_to_text(pdf_path)

    # Procesar cada imagen y su correspondiente texto del PDF
    for idx, image_path in enumerate(image_paths):
        # Procesar la imagen
        image_response = process_image(image_path, report_id, idx + 1)

        # Extraer solo el contenido
        content = image_response.get('choices', [{}])[0].get('message', {}).get('content', "")

        # Preparar un diccionario para almacenar los resultados de la página
        page_result = content

        # Verificar si el contenido contiene "warning"
        if content and "warning" in content.lower():
            # Procesar el texto de la página correspondiente del PDF
            page_content = pdf_text_contents[idx]
            text_response = process_text_with_openai(page_content, report_id, idx + 1, content)
            page_result["text_response"] = text_response  # Guardar el resultado del texto

        # Guardar el resultado de la página en un archivo JSON
        page_result_path = os.path.join(output_dir, f"{report_id}_page_{idx + 1}_result.json")
        with open(page_result_path, "w", encoding="utf-8") as jsonfile:
            json.dump(page_result, jsonfile, indent=4)

        print(f"Resultados de la página {idx + 1} guardados en {page_result_path}")



# Ruta al PDF y directorio de salida para las imágenes
pdf_path = r'C:\Users\jon.ruizcarrillo\Desktop\Projects\TFM\inno-dashboard-assistant-2024\Data_Transformation\OpenAI_API\Data_Structuring\sample_PDFs\Eagle_RRHH.pdf'
output_dir = r'C:\Users\jon.ruizcarrillo\Desktop\Projects\TFM\inno-dashboard-assistant-2024\Data_Transformation\OpenAI_API\Data_Structuring\sample_json_reports'

# Ejecutar la función principal
main(pdf_path, output_dir)


Resultados de la página 1 guardados en C:\Users\jon.ruizcarrillo\Desktop\Projects\TFM\inno-dashboard-assistant-2024\Data_Transformation\OpenAI_API\Data_Structuring\sample_json_reports\Eagle_RRHH_page_1_result.json
Resultados de la página 2 guardados en C:\Users\jon.ruizcarrillo\Desktop\Projects\TFM\inno-dashboard-assistant-2024\Data_Transformation\OpenAI_API\Data_Structuring\sample_json_reports\Eagle_RRHH_page_2_result.json
Resultados de la página 3 guardados en C:\Users\jon.ruizcarrillo\Desktop\Projects\TFM\inno-dashboard-assistant-2024\Data_Transformation\OpenAI_API\Data_Structuring\sample_json_reports\Eagle_RRHH_page_3_result.json
Resultados de la página 4 guardados en C:\Users\jon.ruizcarrillo\Desktop\Projects\TFM\inno-dashboard-assistant-2024\Data_Transformation\OpenAI_API\Data_Structuring\sample_json_reports\Eagle_RRHH_page_4_result.json
Resultados de la página 5 guardados en C:\Users\jon.ruizcarrillo\Desktop\Projects\TFM\inno-dashboard-assistant-2024\Data_Transformation\OpenAI_A

TypeError: 'str' object does not support item assignment